In [ ]:
import os, glob
import numpy as np
import pandas as pd
from datetime import time
from pytz import timezone
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [ ]:
# 1) —— Data Import & Preprocessing —— #

DATA_DIR = "../SPX"
OUTPUT_FEATURE_CSV = "../csvfiles_python/features_360_raw.csv"
OUTPUT_ALLSET_CSV  = "../csvfiles_python/allSet_360_raw.csv"

def load_data(path):
    files = glob.glob(os.path.join(path, "*.txt"))
    dfs = []
    for f in sorted(files):
        df = pd.read_csv(f, names=["DateTime","Open","High","Low","Close"],
                         sep=",", parse_dates=["DateTime"])
        dfs.append(df)
    spx = pd.concat(dfs, ignore_index=True)
    # localize to EST
    spx["DateTime"] = spx["DateTime"].dt.tz_localize(timezone("US/Eastern"))
    spx.set_index("DateTime", inplace=True)
    # filter trading hours 9:30–16:00
    return spx.between_time("09:30","16:00")

spx = load_data(DATA_DIR)

# Drop duplicates
spx = spx[~spx.index.duplicated(keep='first')]

In [ ]:
# 2) —— Create daily labels —— #

# group by calendar date
spx["Date"] = spx.index.date
groups = spx.groupby("Date")

# last-minute close price per day
lmP = groups["Close"].last()

# average price from start to (end − 30 mins)
avgP = groups.apply(lambda df: df["Close"].iloc[:-30].mean())

# binary label: 1 if avgP < lmP else 0
y = (avgP < lmP).astype(int)
y.name = "Y"

In [ ]:
# 3) —— Technical‐Indicator Functions —— #

# Basic EMAs & SMAs
def SMA(x, n):
    return x.rolling(n, min_periods=n).mean()

def EMA(x, n):
    return x.ewm(span=n, adjust=False).mean()

def DEMA(x, n):
    e = EMA(x, n)
    return 2*e - EMA(e, n)

# True Range & ATR
def ATR(H, L, C, n=14):
    prevC = C.shift(1)
    tr = pd.concat([
        H - L,
        (H - prevC).abs(),
        (L - prevC).abs()
    ], axis=1).max(axis=1)
    return EMA(tr, n)

# Average Directional Index (ADX)
def ADX(H, L, C, n=14):
    up = H.diff()
    down = -L.diff()
    plusDM  = np.where((up>down)&(up>0), up, 0.0)
    minusDM = np.where((down>up)&(down>0), down, 0.0)
    tr = pd.concat([
        H-L,
        (H - C.shift(1)).abs(),
        (L - C.shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = EMA(tr, n)
    plusDI  = 100 * EMA(pd.Series(plusDM, index=H.index), n) / atr
    minusDI = 100 * EMA(pd.Series(minusDM, index=H.index), n) / atr
    dx = 100 * ( (plusDI - minusDI).abs() / (plusDI + minusDI) )
    return EMA(dx, n)

In [ ]:
# Aroon
def aroon(xH, xL, n=14):
    def _aroon_up(series):
        idx = series.argmax()
        return ((n - (len(series)-1 - idx)) / n) * 100
    def _aroon_dn(series):
        idx = series.argmin()
        return ((n - (len(series)-1 - idx)) / n) * 100

    au = xH.rolling(n).apply(_aroon_up, raw=True)
    ad = xL.rolling(n).apply(_aroon_dn, raw=True)
    return pd.DataFrame({"aroonUp": au, "aroonDn": ad})

# Bollinger Bands
def BBands(x, n=20, k=2):
    m = SMA(x, n)
    sd = x.rolling(n).std()
    return pd.DataFrame({
        "bb_up":   m + k*sd,
        "bb_mid":  m,
        "bb_low":  m - k*sd
    })

# Commodity Channel Index
def CCI(H, L, C, n=20):
    TP = (H + L + C) / 3
    M  = SMA(TP, n)
    MD = TP.rolling(n).apply(lambda s: np.mean(np.abs(s - s.mean())), raw=True)
    return (TP - M) / (0.015 * MD)

# Chaikin Volatility
def chaikin_volatility(H, L, n=10, ema_n=10):
    hl = H - L
    e1 = EMA(hl, ema_n)
    e2 = e1.shift(n)
    return (e1 - e2) / e2 * 100

# Close Location Value
def CLV(H, L, C):
    return ((C - L) - (H - C)) / (H - L)

In [ ]:
# Chande Momentum Oscillator
def CMO(x, n=14):
    diff = x.diff()
    up = diff.where(diff>0, 0.0).rolling(n).sum()
    dn = (-diff).where(diff<0, 0.0).rolling(n).sum()
    return 100 * (up - dn) / (up + dn)

# Centre of Gravity Indicator (CTI)
#  TTR’s CTI uses n=10 by default: CTI = (C - SMA(C,n)) / SMA(abs(C - SMA(C,n)),n)
def CTI(C, n=10):
    m = SMA(C, n)
    dev = (C - m).abs()
    return (C - m) / SMA(dev, n)

# Donchian Channel
def DonchianChannel(H, L, n=20):
    up = H.rolling(n).max()
    dn = L.rolling(n).min()
    mid = (up + dn) / 2
    return pd.DataFrame({"dc_up": up, "dc_mid": mid, "dc_dn": dn})

# Detrended Price Oscillator
def DPO(x, n=20):
    m = SMA(x, n)
    shift = int(n/2 + 1)
    return x.shift(shift) - m

# Directional Volatility Index (DVI)
#  TTR’s DVI default n=14: DVI = ROC of range
def DVI(C, n=14):
    rng = C.diff().abs()
    prev = rng.shift(n)

    # safe division: if prev==0 → NaN, else compute as usual
    dvi = (rng - prev) / prev * 100
    dvi = dvi.where(prev != 0)   # sets those inf entries to NaN

    return dvi

In [ ]:
# Guppy Multiple Moving Average
def GMMA(x):
    short = [3,5,8,10,12,15]
    long  = [30,35,40,45,50,60]
    df = {}
    for i in short: df[f"gmma_s{i}"] = EMA(x, i)
    for i in long:  df[f"gmma_l{i}"] = EMA(x, i)
    return pd.DataFrame(df)

# Know Sure Thing
def KST(C, r1=10, r2=15, r3=20, r4=30, 
        n1=10, n2=10, n3=10, n4=15):
    roc1 = C.diff(r1)/C.shift(r1)
    roc2 = C.diff(r2)/C.shift(r2)
    roc3 = C.diff(r3)/C.shift(r3)
    roc4 = C.diff(r4)/C.shift(r4)
    kst = (SMA(roc1, n1)*1 +
           SMA(roc2, n2)*2 +
           SMA(roc3, n3)*3 +
           SMA(roc4, n4)*4)
    signal = SMA(kst, 9)
    return pd.DataFrame({"kst": kst, "signal": signal})

# Lagged differences
def lags(H, L, C, n=1):
    # show lagged close differences by default
    return C.diff(n)

# MACD
def MACD(C, n_fast=12, n_slow=26, n_sig=9):
    fast = EMA(C, n_fast)
    slow = EMA(C, n_slow)
    macd = fast - slow
    sig  = EMA(macd, n_sig)
    hist = macd - sig
    return pd.DataFrame({"macd": macd, "signal": sig, "hist": hist})

In [ ]:
# Price Bands (percent)
def PBands(C, n=20, pct=0.025):
    m = SMA(C, n)
    return pd.DataFrame({
        "pb_up": m*(1+pct),
        "pb_mid": m,
        "pb_dn": m*(1-pct)
    })

# Rate of Change
def ROC(C, n=1):
    return (C - C.shift(n)) / C.shift(n) * 100

# Momentum
def momentum(C, n=1):
    return C - C.shift(n)

# Relative Strength Index
def RSI(C, n=14):
    diff = C.diff()
    up = diff.where(diff>0, 0.0)
    dn = -diff.where(diff<0, 0.0)
    ma_up = EMA(up, n)
    ma_dn = EMA(dn, n)
    rs = ma_up / ma_dn
    return 100 - (100 / (1 + rs))

# Running stats
def runSum(C, n=10):    return C.rolling(n).sum()
def runMin(C, n=10):    return C.rolling(n).min()
def runMax(C, n=10):    return C.rolling(n).max()
def runMedian(C,n=10):  return C.rolling(n).median()

In [ ]:
# Parabolic SAR
def SAR(H, L, af0=0.02, af_step=0.02, af_max=0.2):
    up = True
    af = af0
    ep = H[0]
    sar = [L.iloc[0]]
    for i in range(1, len(H)):
        prev_sar = sar[-1]
        if up:
            new_sar = prev_sar + af*(ep - prev_sar)
            new_sar = min(new_sar, L.iloc[i-1], L.iloc[i])
            if H.iloc[i] > ep:
                ep = H.iloc[i]; af = min(af+af_step, af_max)
            if L.iloc[i] < new_sar:
                up = False; sar.append(ep); ep = L.iloc[i]; af = af0
            else:
                sar.append(new_sar)
        else:
            new_sar = prev_sar + af*(ep - prev_sar)
            new_sar = max(new_sar, H.iloc[i-1], H.iloc[i])
            if L.iloc[i] < ep:
                ep = L.iloc[i]; af = min(af+af_step, af_max)
            if H.iloc[i] > new_sar:
                up = True; sar.append(ep); ep = H.iloc[i]; af = af0
            else:
                sar.append(new_sar)
    return pd.Series(sar, index=H.index)

# Volatility (rolling stdev of log‐returns)
def volatility(H, L, C, n=10):
    lr = np.log(C / C.shift(1))
    return lr.rolling(n).std() * np.sqrt(252* (390/ (16*60)))  # annualize on minute data

# Ultimate Oscillator
def ultimate_oscillator(H, L, C, s1=7, s2=14, s3=28, w1=4, w2=2, w3=1):
    bp = C - np.minimum(L, C.shift(1))
    tr = np.maximum(H - L, np.maximum((H-C.shift(1)).abs(), (L-C.shift(1)).abs()))
    avg1 = bp.rolling(s1).sum() / tr.rolling(s1).sum()
    avg2 = bp.rolling(s2).sum() / tr.rolling(s2).sum()
    avg3 = bp.rolling(s3).sum() / tr.rolling(s3).sum()
    uo = 100 * (w1*avg1 + w2*avg2 + w3*avg3) / (w1+w2+w3)
    return uo

In [ ]:
# Vertical–Horizontal Filter
def VHF(C, n=28):
    num = C.rolling(n).max() - C.rolling(n).min()
    den = C.diff().abs().rolling(n).sum()
    return num / den

# Williams Accumulation/Distribution
def williamsAD(H, L, C):
    clv = ((C - L) - (H - C)) / (H - L)
    return clv  # as in TTR

# Williams %R
def WPR(H, L, C, n=14):
    highest = H.rolling(n).max()
    lowest = L.rolling(n).min()
    return (highest - C) / (highest - lowest) * -100

# ZigZag
def ZigZag(H, L, C, pct=0.05):
    zz = [np.nan] * len(C)
    last_pivot = C.iloc[0]
    last_dir = None

    for i in range(1, len(C)):
        change = (C.iloc[i] - last_pivot) / last_pivot
        if last_dir is None:
            if abs(change) > pct:
                last_dir = np.sign(change)
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
        else:
            if last_dir > 0 and C.iloc[i] > last_pivot:
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
            elif last_dir < 0 and C.iloc[i] < last_pivot:
                last_pivot = C.iloc[i]
                zz[i] = last_pivot
            # reversal?
            elif (last_dir > 0 and (C.iloc[i] - last_pivot) / last_pivot < -pct) or \
                 (last_dir < 0 and (C.iloc[i] - last_pivot) / last_pivot > pct):
                last_dir *= -1
                last_pivot = C.iloc[i]
                zz[i] = last_pivot

    return pd.Series(zz, index=C.index)

In [ ]:
# --- Triple Exponential Average oscillator (TRIX) ---
def TRIX(C, n=9):
    ema1 = EMA(C, n)
    ema2 = EMA(ema1, n)
    ema3 = EMA(ema2, n)
    trix = ema3.pct_change() * 100
    return trix

# --- Traders Dynamic Index (TDI) ---
def TDI(C, n_rsi=13, n_sig=2, n_bb=34, sd=1.618):
    rsi = RSI(C, n_rsi)
    signal = SMA(rsi, n_sig)
    bb    = BBands(rsi, n_bb, sd)
    return pd.DataFrame({
        "rsi":    rsi,
        "signal": signal,
        "bb_up":  bb["bb_up"],
        "bb_mid": bb["bb_mid"],
        "bb_dn":  bb["bb_low"]
    })

# --- Stochastic Momentum Index (SMI) ---
def SMI(H, L, C, n=14, n_fast=3, n_slow=25, n_sig=9):
    
    # median price and half‐range
    m = (H.rolling(n).max() + L.rolling(n).min()) / 2
    d = (H.rolling(n).max() - L.rolling(n).min()) / 2

    # double‐smoothed numerator and denominator
    num = EMA(EMA(C - m, n_fast), n_slow)
    den = EMA(EMA(d,       n_fast), n_slow)

    smi    = 100 * (num / den)
    signal = EMA(smi, n_sig)

    return pd.DataFrame({"smi": smi, "signal": signal})